In [ ]:
!pip install transformers datasets pandas scikit-learn

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer

# Veri seti yükleme
# Trendyol yorum veri setini Kaggle'dan indirip Colab'a yüklemiş olacaksın.
# Örnek: trendyol_comments.csv

# Try reading with a different delimiter, e.g., semicolon
df = pd.read_csv("/content/drive/MyDrive/AI_engineer_IBM/data/e-ticaret_urun_yorumlari.csv", sep=';')

print("Toplam satır:", len(df))
df.head(10)

Toplam satır: 15170


,Metin,Durum
0,evet anlatıldığı gibi,1
1,Daha öncede almıştım bu cihazdan ense ve sakal...,1
2,Ürün gayet başarılı sakal kesmede başlık sayıs...,1
3,Daha öncede aynısını almıştım çok güzel ve kal...,1
4,Erkek kuaförüyüm ense ve sıfır sakal traşı içi...,1
5,ürün gerçekten çok güzel,1
6,Ürün beklediğimden güzel çıktı gayet kullanışl...,1
7,güzel makina tavsiye ederim,1
8,tavsiye edebileceğim çok güzel bir makina,1
9,ürün geldiğinde şarjı vardı. ilk lullanım önce...,1


In [ ]:
# Diyelim ki sınıflar şöyle:
# 0: Olumsuz
# 1: Olumlu
# 2: Nötr
# 3: Kargo şikayeti
# 4: Satıcı şikayeti

df = df.rename(columns={"Metin": "review", "Durum": "target"})
df = df.dropna(subset=["review", "target"])
df.head(10)


,review,target
0,evet anlatıldığı gibi,1
1,Daha öncede almıştım bu cihazdan ense ve sakal...,1
2,Ürün gayet başarılı sakal kesmede başlık sayıs...,1
3,Daha öncede aynısını almıştım çok güzel ve kal...,1
4,Erkek kuaförüyüm ense ve sıfır sakal traşı içi...,1
5,ürün gerçekten çok güzel,1
6,Ürün beklediğimden güzel çıktı gayet kullanışl...,1
7,güzel makina tavsiye ederim,1
8,tavsiye edebileceğim çok güzel bir makina,1
9,ürün geldiğinde şarjı vardı. ilk lullanım önce...,1


In [ ]:
print(df["target"].value_counts())


target
0    6978
1    6799
2    1393
Name: count, dtype: int64


In [ ]:
label_map = {label: i for i, label in enumerate(df["target"].unique())}
label_map

{np.int64(1): 0, np.int64(0): 1, np.int64(2): 2}

In [ ]:
df["target"] = df["target"].map(label_map)
df.head(150)

,review,target
0,evet anlatıldığı gibi,1
1,Daha öncede almıştım bu cihazdan ense ve sakal...,1
2,Ürün gayet başarılı sakal kesmede başlık sayıs...,1
3,Daha öncede aynısını almıştım çok güzel ve kal...,1
4,Erkek kuaförüyüm ense ve sıfır sakal traşı içi...,1
...,...,...
145,cok kaliteli buldum tavsiye ederim,1
146,"çok ince ağız yapısı var, şarjı çok iyi. Braun...",1
147,çok kaliteli herkes e tavsiye ederim,1
148,Traş makinası çok güzel gerçekten mükemmel 3.y...,1


In [ ]:
from transformers import AutoTokenizer

model_name = "dbmdz/bert-base-turkish-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# İlk yorumu token’la
sample = df["review"].iloc[0]
tokenizer(sample)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

{'input_ids': [2, 7027, 26528, 2200, 3], 'token_type_ids': [0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1]}

In [ ]:
from datasets import Dataset

dataset = Dataset.from_pandas(df)

def tokenize_fn(batch):
    enc = tokenizer(
        batch["review"],
        truncation=True,
        padding="max_length",
        max_length=128
    )
    enc["labels"] = batch["target"]  # Trainer'ın istediği kısım bu
    return enc

tokenized_ds = dataset.map(tokenize_fn, batched=True)



Map:   0%|          | 0/15170 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

num_labels = 3  # 0: olumsuz, 1: olumlu, 2: nötr

model = AutoModelForSequenceClassification.from_pretrained(
    "dbmdz/bert-base-turkish-cased",
    num_labels=num_labels
)

# Train-Test Split
tokenized_ds = tokenized_ds.train_test_split(test_size=0.2)

train_ds = tokenized_ds["train"]
test_ds = tokenized_ds["test"]

# Değerlendirme fonksiyonu
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)

    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="macro")

    return {"accuracy": acc, "f1": f1}

# Training ayarları
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/AI_engineer_IBM/model_content",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=20,
    load_best_model_at_end=True,
    report_to="none"

)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-1910751858.py:44: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.214300,0.217929,0.922215,0.854646
2,0.123600,0.287802,0.916612,0.825826
3,0.094600,0.277084,0.923863,0.844088


TrainOutput(global_step=2277, training_loss=0.21132302488420007, metrics={'train_runtime': 822.0585, 'train_samples_per_second': 44.289, 'train_steps_per_second': 2.77, 'total_flos': 2394858328160256.0, 'train_loss': 0.21132302488420007, 'epoch': 3.0})

In [ ]:
import torch

sample_text = "Özellikler gayet iyiydi, fakat ürün geldiğinde kalitesi ve kullanışı hiç memnun etmedi beni."
inputs = tokenizer(sample_text, return_tensors="pt", truncation=True, padding=True)


device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
inputs = {k: v.to(device) for k, v in inputs.items()}

outputs = model(**inputs)
pred_label = outputs.logits.argmax(dim=1).item()

print("Tahmin:", pred_label)

Tahmin: 0


In [ ]:
from transformers import BertForSequenceClassification, AutoTokenizer

# Tokenizer aynı şekilde yüklenir
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased")

# Eğitilmiş modeli yükle
model = BertForSequenceClassification.from_pretrained("/content/drive/MyDrive/AI_engineer_IBM/model_content/checkpoint-759")
model.eval()  # Değerlendirme moduna al

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

sample_text = "Özellikler gayet iyiydi, fakat ürün geldiğinde kalitesi ve kullanışı hiç memnun etmedi beni."
inputs = tokenizer(sample_text, return_tensors="pt", truncation=True, padding=True)
inputs = {k: v.to(device) for k, v in inputs.items()}

outputs = model(**inputs)
pred_label = outputs.logits.argmax(dim=1).item()
print("Tahmin:", pred_label)

Tahmin: 0
